# LSTM: Long Short-Term Memory Networks

## Understanding LSTM Networks

**Based on the concepts from "Understanding LSTM Networks" by Christopher Olah**

### Introduction

Humans don't start their thinking from scratch every second. As you read this text, you understand each word based on your understanding of previous words. You don't throw everything away and start thinking from scratch again. Your thoughts have **persistence**.

Traditional neural networks can't do this, and it seems like a major shortcoming. **Recurrent Neural Networks** address this issue through loops that allow information to persist. However, vanilla RNNs have a critical limitation: **the vanishing gradient problem**.

**LSTM (Long Short-Term Memory)** networks, introduced by Hochreiter & Schmidhuber in 1997, solve this problem through a clever gating mechanism. They are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their **default behavior**!

### Learning Objectives

1. Understand the vanishing gradient problem in vanilla RNNs
2. Learn LSTM architecture and gating mechanisms step-by-step
3. Understand the "cell state" as a gradient highway
4. Implement LSTM from scratch
5. See how LSTM solves long-term dependencies

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

---

## 1. The Problem of Long-Term Dependencies

### 1.1 When RNNs Work Well

Sometimes, we only need to look at **recent information** to perform the present task.

**Example:** Predicting the next word in "the clouds are in the **___**"

```
┌────────────────────────────────────────────────────────────┐
│              SHORT-TERM DEPENDENCY                         │
└────────────────────────────────────────────────────────────┘

Input:  "the"  "clouds"  "are"  "in"  "the"  ???
         │       │        │      │      │      ↓
        h₀  →  h₁   →   h₂  →  h₃  →  h₄  →  h₅
                                              ↓
                                          Predict: "sky"

✓ Recent context is sufficient!
✓ Vanilla RNN works fine here
```

We don't need any further context – it's pretty obvious the next word is going to be **sky**. In such cases, where the gap between the relevant information and the place that it's needed is **small**, RNNs can learn to use the past information.

### 1.2 When RNNs Fail: Long-Term Dependencies

But there are also cases where we need **more context**.

**Example:** Predicting the last word in "I grew up in France... I speak fluent **___**"

```
┌────────────────────────────────────────────────────────────┐
│              LONG-TERM DEPENDENCY                          │
└────────────────────────────────────────────────────────────┘

Input: "I" "grew" "up" "in" "France" ... "I" "speak" "fluent" ???
        │    │     │    │      │      ...  │     │       │     ↓
       h₀ → h₁ → h₂ → h₃ →  h₄  → ... → h₁₀ → h₁₁ → h₁₂ → h₁₃
                            ↑                                ↓
                      Important info!                  Predict: "French"
                      (but far away)

❌ Recent context suggests "a language"
❌ But which language? Need "France" from way back!
❌ Vanilla RNN struggles with this gap
```

Recent information suggests that the next word is probably the name of a language, but if we want to narrow down **which** language, we need the context of "France", from further back. It's entirely possible for the gap between the relevant information and the point where it is needed to become **very large**.

Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

### 1.3 The Vanishing Gradient Problem

**Why do RNNs fail at long-term dependencies?**

The answer lies in how gradients flow during backpropagation.

```
┌────────────────────────────────────────────────────────────┐
│         GRADIENT FLOW IN VANILLA RNN                       │
└────────────────────────────────────────────────────────────┘

Forward Pass (Information Flow):
t=0 → t=1 → t=2 → t=3 → t=4 → t=5 → t=6 → t=7 → t=8 → t=9
│     │     │     │     │     │     │     │     │     │
h₀ → h₁ → h₂ → h₃ → h₄ → h₅ → h₆ → h₇ → h₈ → h₉ → Loss

Backward Pass (Gradient Flow):
t=9 ← t=8 ← t=7 ← t=6 ← t=5 ← t=4 ← t=3 ← t=2 ← t=1 ← t=0
│     │     │     │     │     │     │     │     │     │
████  ███   ██    █     ▁     ▁     ▁     ▁     ▁     ▁
100%  75%   50%   25%   10%   5%    2%    1%    0.5%  0.1%

Problem: Gradient magnitude decays exponentially!
         Early time steps receive almost no gradient signal.
```

#### Mathematical Explanation

In vanilla RNN, the gradient flows through repeated matrix multiplications:

$$\frac{\partial L}{\partial h_0} = \frac{\partial L}{\partial h_T} \cdot \prod_{t=1}^{T} \frac{\partial h_t}{\partial h_{t-1}}$$

Each term $\frac{\partial h_t}{\partial h_{t-1}}$ involves:
- Matrix multiplication by $W_{hh}$
- Derivative of activation function (tanh)

**Key insight:**
- If $|W_{hh}| < 1$ and $|\text{tanh}'| < 1$, the product shrinks exponentially
- After T steps: gradient $\approx (0.7)^T \cdot \text{original gradient}$
- For T=10: gradient is only **2.8%** of original
- For T=20: gradient is only **0.08%** of original

**Consequence:** The network can't learn long-term dependencies!

In [ ]:
# Simulate gradient decay
def simulate_gradient_decay(T, decay_rate=0.7):
    """Simulate how gradient decays over time steps."""
    gradients = []
    gradient = 1.0  # Start with gradient of 1.0
    
    for t in range(T):
        gradients.append(gradient)
        gradient *= decay_rate  # Multiply by decay rate
    
    return np.array(gradients[::-1])  # Reverse (backward pass)

# Simulate for different sequence lengths
T_values = [10, 20, 30]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, T in enumerate(T_values):
    gradients = simulate_gradient_decay(T)
    time_steps = np.arange(T)
    
    axes[idx].bar(time_steps, gradients, alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('Time Step')
    axes[idx].set_ylabel('Gradient Magnitude')
    axes[idx].set_title(f'Gradient Decay (T={T})')
    axes[idx].set_ylim([0, 1.1])
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Annotate first and last
    axes[idx].text(0, gradients[0] + 0.05, f'{gradients[0]:.2f}', 
                  ha='center', fontweight='bold')
    axes[idx].text(T-1, gradients[-1] + 0.05, f'{gradients[-1]:.4f}', 
                  ha='center', fontweight='bold', color='red')

plt.tight_layout()
plt.show()

print("\n📊 Gradient Decay Analysis:")
for T in T_values:
    final_gradient = 0.7 ** T
    print(f"  T={T:2d}: Final gradient = {final_gradient:.6f} ({final_gradient*100:.2f}% of original)")

print("\n❌ Problem: Early time steps receive almost no learning signal!")

---

## 2. LSTM Networks: The Solution

### 2.1 The Core Idea: Cell State

The key to LSTMs is the **cell state** – the horizontal line running through the top of the diagram.

```
┌────────────────────────────────────────────────────────────┐
│              THE CELL STATE: GRADIENT HIGHWAY              │
└────────────────────────────────────────────────────────────┘

                C_{t-1} ──────────────→ C_t ──────────────→ C_{t+1}
                  ↑                      ↑                     ↑
            (cell state)           (cell state)          (cell state)

The cell state is kind of like a conveyor belt.
It runs straight down the entire chain, with only some minor
linear interactions. It's very easy for information to just
flow along it unchanged.

This is the "gradient highway" that solves vanishing gradient!
```

The cell state is kind of like a **conveyor belt**. It runs straight down the entire chain, with only some minor linear interactions. It's very easy for information to just flow along it unchanged.

The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called **gates**.

### 2.2 Gates: Controlling Information Flow

Gates are a way to optionally let information through. They are composed of:
1. A **sigmoid neural net layer** (outputs 0 to 1)
2. A **pointwise multiplication operation**

```
┌────────────────────────────────────────────────────────────┐
│                  HOW GATES WORK                            │
└────────────────────────────────────────────────────────────┘

Sigmoid Layer:
  σ(x) → outputs between 0 and 1
  
  0 = "let nothing through"
  1 = "let everything through"
  0.5 = "let half through"

Example:
  Information: [0.8, -0.3, 0.5, 0.2]
  Gate:        [1.0,  0.0, 0.5, 0.8]
               ×     ×     ×     ×
  Result:      [0.8,  0.0, 0.25, 0.16]
                ↑     ↑     ↑     ↑
              keep  block  half  most
```

An LSTM has **three of these gates**, to protect and control the cell state:
1. **Forget Gate** - What to forget from cell state
2. **Input Gate** - What new information to add
3. **Output Gate** - What to output from cell state

### 2.3 Complete LSTM Architecture

```
┌────────────────────────────────────────────────────────────┐
│                    LSTM CELL STRUCTURE                     │
└────────────────────────────────────────────────────────────┘

Inputs: x_t (current input), h_{t-1} (previous hidden), C_{t-1} (previous cell)

                    ┌─────────────────┐
                    │  FORGET GATE    │
                    │  f_t = σ(...)   │  ← Step 1: What to forget
                    └────────┬────────┘
                             │
    C_{t-1} ────────────────×────────────────┐
    (old cell)               │                │
                             │                │
                    ┌────────▼────────┐       │
                    │  INPUT GATE     │       │
                    │  i_t = σ(...)   │  ← Step 2: What to add
                    └────────┬────────┘       │
                             │                │
                    ┌────────▼────────┐       │
                    │  CELL GATE      │       │
                    │  C̃_t = tanh(...) │  ← Step 2: Candidate values
                    └────────┬────────┘       │
                             │                │
                             ×────────────────┤
                             │                │
                             ▼                ▼
                            C_t ──────────────┐  ← Step 3: Update cell
                        (new cell)            │
                             │                │
                    ┌────────▼────────┐       │
                    │  OUTPUT GATE    │       │
                    │  o_t = σ(...)   │  ← Step 4: What to output
                    └────────┬────────┘       │
                             │                │
                             ×────────────────┘
                             │
                             ▼
                            h_t
                      (new hidden)

Legend:
  σ = sigmoid (0 to 1) - acts as a gate
  × = element-wise multiplication
  tanh = hyperbolic tangent (-1 to 1)
  + = element-wise addition
```

---

## 3. Step-by-Step LSTM Walk Through

Let's walk through each step of the LSTM with intuitive examples.

### Step 1: Forget Gate Layer

The first step in our LSTM is to decide what information we're going to **throw away** from the cell state.

```
f_t = σ(W_f · [h_{t-1}, x_t] + b_f)
```

**What it does:**
- Looks at $h_{t-1}$ and $x_t$
- Outputs a number between 0 and 1 for each number in the cell state $C_{t-1}$
- **1** = "completely keep this"
- **0** = "completely get rid of this"

**Example:** Language modeling

```
┌────────────────────────────────────────────────────────────┐
│              FORGET GATE EXAMPLE                           │
└────────────────────────────────────────────────────────────┘

Sentence: "The cat sat on the mat. The dog ran."
                                      ↑
                              New sentence starts!

Cell state might include:
  • Gender of subject ("cat" = neutral/feminine)
  • Number (singular)
  • Tense (past)

When we see "The dog":
  Forget gate → 0 for gender of "cat"
  (We want to forget the old subject's gender)
```

### Step 2: Input Gate Layer

The next step is to decide what **new information** we're going to store in the cell state.

This has two parts:

```
i_t = σ(W_i · [h_{t-1}, x_t] + b_i)      ← Input gate: what to update
C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)   ← Candidate values: what to add
```

**What it does:**
- **Input gate** ($i_t$): Decides which values we'll update (0 to 1)
- **Candidate values** ($\tilde{C}_t$): Creates new candidate values (-1 to 1)

**Example:** Language modeling (continued)

```
┌────────────────────────────────────────────────────────────┐
│              INPUT GATE EXAMPLE                            │
└────────────────────────────────────────────────────────────┘

Sentence: "The cat sat on the mat. The dog ran."
                                      ↑
                              New subject: "dog"

We want to add:
  • Gender of new subject ("dog" = masculine/neutral)
  • Number (singular)
  • Tense (past)

Input gate → 1 for gender dimension
Candidate → value representing "dog's" gender
```

### Step 3: Update Cell State

It's now time to update the old cell state, $C_{t-1}$, into the new cell state $C_t$.

```
C_t = f_t ⊙ C_{t-1} + i_t ⊙ C̃_t
      ↑              ↑
   forget old    add new
```

**What it does:**
- Multiply the old state by $f_t$, forgetting the things we decided to forget
- Add $i_t \odot \tilde{C}_t$, the new candidate values scaled by how much we decided to update

**This is the gradient highway!**

```
┌────────────────────────────────────────────────────────────┐
│              CELL STATE UPDATE                             │
└────────────────────────────────────────────────────────────┘

Old cell state (C_{t-1}):
  [gender: cat, number: singular, tense: past]

Forget gate (f_t):
  [0, 1, 1]  ← Forget gender, keep number and tense

Input gate (i_t):
  [1, 0, 0]  ← Add new gender, don't change others

Candidate (C̃_t):
  [gender: dog, -, -]

New cell state (C_t):
  [gender: dog, number: singular, tense: past]
   ↑            ↑                  ↑
  updated      kept               kept
```

### Step 4: Output Gate Layer

Finally, we need to decide what we're going to **output**.

```
o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
h_t = o_t ⊙ tanh(C_t)
```

**What it does:**
- Output gate decides what parts of the cell state to output
- We put the cell state through tanh (to push values to -1 to 1)
- Multiply by the output gate, so we only output the parts we decided to

**Example:** Language modeling (continued)

```
┌────────────────────────────────────────────────────────────┐
│              OUTPUT GATE EXAMPLE                           │
└────────────────────────────────────────────────────────────┘

Sentence: "The dog ran"
                    ↑
            Just saw subject, what's next?

Cell state contains:
  [gender: dog, number: singular, tense: past]

Since we just saw a subject, we might want to output
information relevant to a VERB:
  • Number (singular) → verb should be "runs" not "run"
  • Tense (past) → verb should be "ran" not "runs"

Output gate → 1 for number and tense
Output gate → 0 for gender (not relevant for verb)
```

### 3.1 Complete LSTM Equations

Putting it all together:

$$
\begin{align}
f_t &= \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) && \text{(Forget gate)} \\
i_t &= \sigma(W_i \cdot [h_{t-1}, x_t] + b_i) && \text{(Input gate)} \\
\tilde{C}_t &= \tanh(W_C \cdot [h_{t-1}, x_t] + b_C) && \text{(Candidate values)} \\
C_t &= f_t \odot C_{t-1} + i_t \odot \tilde{C}_t && \text{(Update cell state)} \\
o_t &= \sigma(W_o \cdot [h_{t-1}, x_t] + b_o) && \text{(Output gate)} \\
h_t &= o_t \odot \tanh(C_t) && \text{(Output hidden state)}
\end{align}
$$

---

## 4. Why LSTM Solves Vanishing Gradient

### 4.1 The Gradient Highway

```
┌────────────────────────────────────────────────────────────┐
│         GRADIENT FLOW: RNN vs LSTM                         │
└────────────────────────────────────────────────────────────┘

Vanilla RNN:
  ∂L/∂h_0 = ∂L/∂h_T · ∏(W_hh · tanh')  ← Repeated multiplication
                       ↑
                  Causes decay!

LSTM:
  ∂L/∂C_0 = ∂L/∂C_T · ∏(f_t)  ← Mostly addition through cell state
                       ↑
                  Gradient highway!

Key Difference:
• RNN: Gradient flows through repeated matrix multiplications
• LSTM: Gradient flows through addition (cell state update)
• Addition preserves gradient magnitude!

Cell State Update:
  C_t = f_t ⊙ C_{t-1} + i_t ⊙ C̃_t
        ↑
  When f_t ≈ 1, gradient flows through unchanged!

Result:
  RNN:  Gradient decays exponentially (0.7^T)
  LSTM: Gradient can flow unchanged through cell state!
```

---

## 5. LSTM Implementation

In [ ]:
def sigmoid(x):
    """Sigmoid activation: σ(x) = 1 / (1 + e^(-x))"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def tanh(x):
    """Hyperbolic tangent activation."""
    return np.tanh(x)

def softmax(x):
    """Softmax activation."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

print("✓ Activation functions defined")

In [ ]:
class LSTM:
    """LSTM implementation from scratch."""
    
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.001):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        
        # Initialize weights for forget gate
        self.W_f = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_f = np.zeros((hidden_dim, 1))
        
        # Initialize weights for input gate
        self.W_i = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_i = np.zeros((hidden_dim, 1))
        
        # Initialize weights for cell gate
        self.W_C = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_C = np.zeros((hidden_dim, 1))
        
        # Initialize weights for output gate
        self.W_o = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_o = np.zeros((hidden_dim, 1))
        
        # Initialize weights for output layer
        self.W_y = np.random.randn(output_dim, hidden_dim) * 0.01
        self.b_y = np.zeros((output_dim, 1))
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
    
    def forward(self, X):
        """Forward pass through LSTM.
        
        Args:
            X: Input sequence (seq_length, input_dim)
        
        Returns:
            outputs: List of output probabilities
            cache: Dictionary containing intermediate values for backprop
        """
        seq_length = X.shape[0]
        
        # Initialize hidden state and cell state
        h = np.zeros((self.hidden_dim, 1))
        C = np.zeros((self.hidden_dim, 1))
        
        # Store values for backprop
        cache = {
            'h': [h],
            'C': [C],
            'f': [],
            'i': [],
            'C_tilde': [],
            'o': [],
            'concat': []
        }
        
        outputs = []
        
        for t in range(seq_length):
            # Get current input
            x_t = X[t].reshape(-1, 1)
            
            # Concatenate h and x
            concat = np.vstack([h, x_t])
            cache['concat'].append(concat)
            
            # Forget gate: decides what to forget from cell state
            f_t = sigmoid(np.dot(self.W_f, concat) + self.b_f)
            cache['f'].append(f_t)
            
            # Input gate: decides what new information to add
            i_t = sigmoid(np.dot(self.W_i, concat) + self.b_i)
            cache['i'].append(i_t)
            
            # Cell gate: creates candidate values
            C_tilde_t = tanh(np.dot(self.W_C, concat) + self.b_C)
            cache['C_tilde'].append(C_tilde_t)
            
            # Update cell state: forget old + add new
            C = f_t * C + i_t * C_tilde_t
            cache['C'].append(C)
            
            # Output gate: decides what to output
            o_t = sigmoid(np.dot(self.W_o, concat) + self.b_o)
            cache['o'].append(o_t)
            
            # Update hidden state
            h = o_t * tanh(C)
            cache['h'].append(h)
            
            # Compute output
            logits = np.dot(self.W_y, h) + self.b_y
            y_t = softmax(logits.flatten())
            outputs.append(y_t)
        
        return outputs, cache
    
    def compute_loss(self, outputs, targets):
        """Compute cross-entropy loss."""
        seq_length = len(outputs)
        loss = 0.0
        for t in range(seq_length):
            loss += -np.log(outputs[t][targets[t]] + 1e-8)
        return loss / seq_length
    
    def backward(self, X, targets, outputs, cache):
        """Backpropagation through time for LSTM."""
        seq_length = X.shape[0]
        
        # Initialize gradients
        dW_f = np.zeros_like(self.W_f)
        db_f = np.zeros_like(self.b_f)
        dW_i = np.zeros_like(self.W_i)
        db_i = np.zeros_like(self.b_i)
        dW_C = np.zeros_like(self.W_C)
        db_C = np.zeros_like(self.b_C)
        dW_o = np.zeros_like(self.W_o)
        db_o = np.zeros_like(self.b_o)
        dW_y = np.zeros_like(self.W_y)
        db_y = np.zeros_like(self.b_y)
        
        dh_next = np.zeros((self.hidden_dim, 1))
        dC_next = np.zeros((self.hidden_dim, 1))
        
        # Backprop through time
        for t in reversed(range(seq_length)):
            # Output layer gradient
            dy = outputs[t].copy()
            dy[targets[t]] -= 1
            dy = dy.reshape(-1, 1) / seq_length
            
            dW_y += np.dot(dy, cache['h'][t+1].T)
            db_y += dy
            
            # Hidden state gradient
            dh = np.dot(self.W_y.T, dy) + dh_next
            
            # Output gate gradient
            do = dh * tanh(cache['C'][t+1])
            do_raw = do * cache['o'][t] * (1 - cache['o'][t])
            
            # Cell state gradient
            dC = dh * cache['o'][t] * (1 - tanh(cache['C'][t+1])**2) + dC_next
            
            # Cell gate gradient
            dC_tilde = dC * cache['i'][t]
            dC_tilde_raw = dC_tilde * (1 - cache['C_tilde'][t]**2)
            
            # Input gate gradient
            di = dC * cache['C_tilde'][t]
            di_raw = di * cache['i'][t] * (1 - cache['i'][t])
            
            # Forget gate gradient
            df = dC * cache['C'][t]
            df_raw = df * cache['f'][t] * (1 - cache['f'][t])
            
            # Accumulate gradients
            dW_f += np.dot(df_raw, cache['concat'][t].T)
            db_f += df_raw
            dW_i += np.dot(di_raw, cache['concat'][t].T)
            db_i += di_raw
            dW_C += np.dot(dC_tilde_raw, cache['concat'][t].T)
            db_C += dC_tilde_raw
            dW_o += np.dot(do_raw, cache['concat'][t].T)
            db_o += do_raw
            
            # Gradient for next iteration
            dconcat = (np.dot(self.W_f.T, df_raw) + 
                      np.dot(self.W_i.T, di_raw) + 
                      np.dot(self.W_C.T, dC_tilde_raw) + 
                      np.dot(self.W_o.T, do_raw))
            
            dh_next = dconcat[:self.hidden_dim, :]
            dC_next = dC * cache['f'][t]
        
        # Gradient clipping
        for grad in [dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y]:
            np.clip(grad, -5, 5, out=grad)
        
        return dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y
    
    def update_parameters(self, grads):
        """Update parameters using gradients."""
        dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y = grads
        
        self.W_f -= self.learning_rate * dW_f
        self.b_f -= self.learning_rate * db_f
        self.W_i -= self.learning_rate * dW_i
        self.b_i -= self.learning_rate * db_i
        self.W_C -= self.learning_rate * dW_C
        self.b_C -= self.learning_rate * db_C
        self.W_o -= self.learning_rate * dW_o
        self.b_o -= self.learning_rate * db_o
        self.W_y -= self.learning_rate * dW_y
        self.b_y -= self.learning_rate * db_y
    
    def train_step(self, X, y):
        """Single training step."""
        outputs, cache = self.forward(X)
        loss = self.compute_loss(outputs, y)
        grads = self.backward(X, y, outputs, cache)
        self.update_parameters(grads)
        return loss, cache
    
    def predict(self, X):
        """Make predictions."""
        outputs, _ = self.forward(X)
        predictions = np.array([np.argmax(out) for out in outputs])
        return predictions
    
    def evaluate(self, X, y):
        """Evaluate on dataset."""
        total_loss = 0.0
        correct = 0
        total = 0
        
        for i in range(len(X)):
            outputs, _ = self.forward(X[i])
            loss = self.compute_loss(outputs, y[i])
            total_loss += loss
            
            predictions = np.array([np.argmax(out) for out in outputs])
            correct += np.sum(predictions == y[i])
            total += len(y[i])
        
        return total_loss / len(X), correct / total

print("✓ LSTM class defined")
print("\n📊 LSTM has 4 gates (forget, input, cell, output) + output layer")
print("   Total parameters: Much more than vanilla RNN!")

---

## 6. Training and Evaluation

In [ ]:
# Create synthetic data for demonstration
n_samples = 100
seq_length = 50
input_dim = 13
n_classes = 10

X_train = np.random.randn(n_samples, seq_length, input_dim)
X_val = np.random.randn(20, seq_length, input_dim)
y_train = np.random.randint(0, n_classes, n_samples)
y_val = np.random.randint(0, n_classes, 20)

print("✓ Synthetic data created for demonstration")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")

In [ ]:
# Initialize LSTM
hidden_dim = 64
output_dim = n_classes

lstm = LSTM(input_dim, hidden_dim, output_dim, learning_rate=0.001)

print("LSTM initialized:")
print(f"  Input dim: {input_dim}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Output dim: {output_dim}")
print(f"\n  Parameters:")
print(f"    Forget gate: {lstm.W_f.size + lstm.b_f.size}")
print(f"    Input gate:  {lstm.W_i.size + lstm.b_i.size}")
print(f"    Cell gate:   {lstm.W_C.size + lstm.b_C.size}")
print(f"    Output gate: {lstm.W_o.size + lstm.b_o.size}")
print(f"    Output layer: {lstm.W_y.size + lstm.b_y.size}")
total_params = (lstm.W_f.size + lstm.b_f.size + lstm.W_i.size + lstm.b_i.size + 
                lstm.W_C.size + lstm.b_C.size + lstm.W_o.size + lstm.b_o.size + 
                lstm.W_y.size + lstm.b_y.size)
print(f"    TOTAL: {total_params}")

In [ ]:
# Training loop
n_epochs = 20
print(f"Training LSTM for {n_epochs} epochs...\n")

for epoch in range(n_epochs):
    # Shuffle training data
    indices = np.random.permutation(len(X_train))
    
    # Train
    for idx in indices:
        seq_label = y_train[idx]
        frame_labels = np.full(X_train[idx].shape[0], seq_label)
        lstm.train_step(X_train[idx], frame_labels)
    
    # Evaluate
    train_loss, train_acc = lstm.evaluate(
        X_train,
        np.array([np.full(X_train[i].shape[0], y_train[i]) for i in range(len(X_train))])
    )
    val_loss, val_acc = lstm.evaluate(
        X_val,
        np.array([np.full(X_val[i].shape[0], y_val[i]) for i in range(len(X_val))])
    )
    
    # Store
    lstm.train_losses.append(train_loss)
    lstm.val_losses.append(val_loss)
    lstm.train_accuracies.append(train_acc)
    lstm.val_accuracies.append(val_acc)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("\n✓ Training complete!")

---

## 7. Summary and Conclusion

### What We Learned:

✅ **Long-term dependencies** - Why vanilla RNNs fail on long sequences  
✅ **LSTM architecture** - 4 gates (forget, input, cell, output) + cell state  
✅ **Cell state as gradient highway** - Allows gradients to flow unchanged  
✅ **Step-by-step walkthrough** - Each gate with intuitive examples  
✅ **Implementation** - Complete LSTM from scratch with BPTT  

### Key Insights:

```
┌────────────────────────────────────────────────────────────┐
│              LSTM: THE BIG PICTURE                         │
└────────────────────────────────────────────────────────────┘

Problem:
  Vanilla RNNs can't learn long-term dependencies
  → Vanishing gradient problem

Solution:
  LSTM introduces:
  1. Cell state (gradient highway)
  2. Gates (control information flow)
  3. Forget gate (what to forget)
  4. Input gate (what to add)
  5. Output gate (what to output)

Result:
  ✓ Can learn dependencies over 100+ steps
  ✓ Gradients flow through cell state unchanged
  ✓ Gates learn when to remember/forget
  ✓ Default behavior is to remember!
```

### Variants and Extensions:

The LSTM we described is a pretty normal LSTM. But not all LSTMs are the same. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but worth mentioning:

1. **Peephole connections** - Let gates look at cell state
2. **Coupled forget and input gates** - Decide together what to forget/add
3. **GRU (Gated Recurrent Unit)** - Simpler variant with only 2 gates

### Next Steps:

In **Notebook 4**, we'll explore:
- **GRU (Gated Recurrent Unit)** - Simpler alternative to LSTM
- **Fewer gates** - Only 2 gates instead of 4
- **Faster training** - Fewer parameters (~25% less)
- **Similar performance** - Often matches LSTM

**LSTM is the workhorse of sequence modeling!** 🎯

---

*This notebook was inspired by Christopher Olah's excellent blog post "Understanding LSTM Networks"*